# Prototype: Interactive map with elevation shading

Current functionality of the prototype:
- Generates a map using Folium/leaflet
- Creates a grid on the map
- Finds the latitude and longditude of all the x,y coordinates in the grid
- Finds the elevation for each of the x,y coordinates (using Open-Elevation API)
- Shades the "boxes" in the grid based on the elevation and if it's within a threshold (IN PROGRESS)

Links:
- Coordinates can be found using e.g. https://www.gps-coordinates.net/
- Open-Elevation API: https://open-elevation.com/

### To-Do
- Look at how the elevation is printed. I believe there is something wrong, as the scrips can perfectly print when the elevation is below the threshold, but not above
- Fix the shading - The shading looks rather random at the moment
- Online, working prototype of the map

In [1]:
# Importing necessary libraries

import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import folium
from folium.plugins import MarkerCluster

# Request libraries to retrive data through API
import requests as re

### Define the center of the map, size of the grid, number of boxes in the grid, and the elevation threshold

In [2]:
# Define the center lat,lon for the map
location = [55.731768, 12.58149]

# Define the upper right and lower left lon, lat you want to span the grid across
lower_left = [55.730731, 12.577041]
upper_right = [55.733814, 12.584928]

# Define the number of boxes in the grid
# BE CAREFUL! The API is extremely slow, so it is suggested to test with two to four boxes
num_boxes = 2

# Find the distance between the x and y coordinates
lat_diff = upper_right[0] - lower_left[0]
lon_diff = upper_right[1] - lower_left[1]

# Define the threshold elevation of where the land is in danger of flooding (60cm)
risk_threshold = 1

# Setting a header in the API request, saying that we are communication in JSON
# This is to ensure that we communicate in JSON if the API do not understand
headers = {'Content-Type': 'application/json',
          'Accept': 'application/json'}

In [3]:
# Define a function that find the lon, lat for a specific box
def lat_lon_from_box(lat_idx, lon_idx):
    
    # The starting point (coordinates) for the grid
    base_lat = lower_left[0]
    base_lon = lower_left[1]
    
    # How many steps have we moved
    moved_lat = lat_idx
    moved_lon = lon_idx
    
    # The "length" in lat/lon of every box (In %)
    # Since the boxes are the same size, we can calculate the size on the steps based on number of boxes
    step = 1 / num_boxes
    
    # Finds the percentage of the whole grid one box is
    lat_increase = lat_diff * (moved_lat * step)
    lon_increase = lon_diff * (moved_lon * step)

    # Finds the coordinate of the specific box
    new_lat = base_lat + lat_increase
    new_lon = base_lon + lon_increase
    return(new_lat, new_lon)

In [4]:
# Testing the above script
lat_lon_from_box(1,2)

(55.7322725, 12.584928)

In [5]:
# Define a function that finds the elevation based on the coordinates from the lat_lon_from_box function
# Uses the Open-Elevation API - https://github.com/Jorl17/open-elevation/blob/master/docs/api.md

def get_elevation(lat, lon):
    api_base = 'https://api.open-elevation.com/api/v1/lookup?locations='
    coord_str = str(lat) + ',' + str(lon)
    res = re.get(api_base + coord_str, None, headers=headers)
    
    # Checking if the request went well
    # The printed results should look like this:
    # Example: {'results': [{'latitude': 56.229, 'elevation': 60, 'longitude': 13.066}]}
    
    if(res.status_code == 200):
        res = res.json()
        res = res['results']
        # The first result
        res = res[0]
        # In the first results, we want to find the elevation field
        res = res['elevation']
        return res
    
    else:
        print("These coordinates did not work: " + coord_str)
        print(res)
    
    # Safe default return
    return 999999

In [6]:
# Testing the above script
# get_elevation(55.731, 12.574)

In [7]:
# Creating the map
# The location is the centerpoint of the map, the zoom tells us how big the map should be
m = folium.Map(location, zoom_start=13)

In [8]:
# Creating a script that plot the boxes

# The script returns a list of geoJSON style object

def get_geojson_grid(upper_right, lower_left, num_boxes):

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], num_boxes+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], num_boxes+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat_idx, lat in enumerate(lat_steps[:-1]):
        for lon_idx, lon in enumerate(lon_steps[:-1]):
            # Find the lat, lon of the specific box
            box_lat, box_lon = lat_lon_from_box(lat_idx, lon_idx)
            # print(box_lat, box_lon)
            
            # Get the specific elevation for the specific box
            elevation = get_elevation(box_lat, box_lon)
            
            # Check if the elevation is under the threshold
            if(elevation <= risk_threshold):
                print('Coordinates: ' + str(box_lat) + ',' + str(box_lon) + ' is to low and in the dangerzone of flooding')
            
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [9]:
#Call the geoJSON function
grid = get_geojson_grid(upper_right, lower_left, num_boxes)

for i, geo_json in enumerate(grid):
    
    # Based on the i (index) in the grid, find the x,y of the specific box (find the box)
    # E.g. 2 turns into 1,2
    # Use the get_lat based on the box index and find the elevation
    # Use this to color the specific box according to the elevation
    # E.g. color = plt.cm.Blues(elevation / len(grid))
    
    row = i / num_boxes
    col = i % num_boxes
    lat, lon = lat_lon_from_box(row, col)
    elevation = get_elevation(lat, lon)
    
    # Another way:
    # blue_color = 1 if elevation <= risk_thershold else 0
    # Tell how much of the color we want: color = plt.cm.Blues(blue_color)
    #get_elevation(box_lat, box_lon)
    color = plt.cm.Reds(elevation / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        # Style the grid
                                                                        # Can use dashArray to create dashes
                                                                        'fillColor': color,
                                                                        'color':"white",
                                                                        'weight': 0.1,
                                                                        'fillOpacity': 0.40,
                                                                    })
    # Cretes a "popup" (infobox), when clicking on a box
    # Instead of i we can put in the elevation
    popup = folium.Popup("Box number {} - Elevation {}".format(i, elevation))
    
    gj.add_child(popup)

    m.add_child(gj)

These coordinates did not work: 55.730731,12.577041
<Response [504]>
These coordinates did not work: 55.73150175,12.5809845
<Response [504]>
These coordinates did not work: 55.7322725,12.577041
<Response [504]>


### Save the map locally

In [10]:
# Saving the map as a html file
# Do not remove ".html"
m.save('gentofte.html')

# The files is located in the same folder as the python file

In [11]:
m

### Ignore this - Previous output

These coordinates did not work: 55.730731,12.5780926
<Response [504]>

These coordinates did not work: 55.730731,12.5786184
<Response [504]>

These coordinates did not work: 55.730731,12.5807216
<Response [504]>

These coordinates did not work: 55.730731,12.581247399999999
<Response [504]>

Coordinates: 55.730731,12.581773199999999 is to low and in the dangerzone of flooding

These coordinates did not work: 55.730731,12.582298999999999
<Response [504]>

These coordinates did not work: 55.730731,12.5828248
<Response [504]>

Coordinates: 55.730731,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.730731,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.730731,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.730936533333335,12.5791442
<Response [504]>

These coordinates did not work: 55.730936533333335,12.5807216
<Response [504]>

Coordinates: 55.730936533333335,12.581773199999999 is to low and in the dangerzone of flooding

These coordinates did not work: 55.730936533333335,12.582298999999999
<Response [504]>

Coordinates: 55.730936533333335,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.730936533333335,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.730936533333335,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.730936533333335,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.73114206666666,12.5775668
<Response [504]>

These coordinates did not work: 55.73114206666666,12.5780926
<Response [504]>

These coordinates did not work: 55.73114206666666,12.5807216
<Response [504]>

Coordinates: 55.73114206666666,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.73114206666666,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.73114206666666,12.5828248 is to low and in the dangerzone of flooding

These coordinates did not work: 55.73114206666666,12.5833506
<Response [504]>

Coordinates: 55.73114206666666,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.73114206666666,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7313476,12.5775668
<Response [502]>

These coordinates did not work: 55.7313476,12.581247399999999
<Response [504]>

Coordinates: 55.7313476,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.7313476,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.7313476,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.7313476,12.5833506 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7313476,12.5838764
<Response [504]>

Coordinates: 55.7313476,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.731553133333335,12.5786184
<Response [504]>

These coordinates did not work: 55.731553133333335,12.57967
<Response [504]>

Coordinates: 55.731553133333335,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.731553133333335,12.582298999999999 is to low and in the dangerzone of flooding

These coordinates did not work: 55.731553133333335,12.5828248
<Response [504]>

Coordinates: 55.731553133333335,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.731553133333335,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.731553133333335,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.731758666666664,12.5791442
<Response [504]>

These coordinates did not work: 55.731758666666664,12.5801958
<Response [504]>

These coordinates did not work: 55.731758666666664,12.581773199999999
<Response [502]>

Coordinates: 55.731758666666664,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.731758666666664,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.731758666666664,12.5833506 is to low and in the dangerzone of flooding

These coordinates did not work: 55.731758666666664,12.5838764
<Response [504]>

Coordinates: 55.731758666666664,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7319642,12.577041
<Response [504]>

These coordinates did not work: 55.7319642,12.5791442
<Response [504]>

These coordinates did not work: 55.7319642,12.5807216
<Response [504]>

These coordinates did not work: 55.7319642,12.581247399999999
<Response [504]>

These coordinates did not work: 55.7319642,12.581773199999999
<Response [504]>

These coordinates did not work: 55.7319642,12.582298999999999
<Response [504]>

Coordinates: 55.7319642,12.5828248 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7319642,12.5833506
<Response [504]>

These coordinates did not work: 55.7319642,12.5838764
<Response [504]>

These coordinates did not work: 55.7319642,12.5844022
<Response [504]>

These coordinates did not work: 55.732169733333336,12.577041
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5780926
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5786184
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5791442
<Response [504]>

These coordinates did not work: 55.732169733333336,12.57967
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5801958
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5807216
<Response [504]>

These coordinates did not work: 55.732169733333336,12.581773199999999
<Response [502]>

Coordinates: 55.732169733333336,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.732169733333336,12.5828248 is to low and in the dangerzone of flooding

These coordinates did not work: 55.732169733333336,12.5833506
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5838764
<Response [504]>

These coordinates did not work: 55.732169733333336,12.5844022
<Response [504]>

These coordinates did not work: 55.732375266666665,12.5775668
<Response [504]>

These coordinates did not work: 55.732375266666665,12.5786184
<Response [504]>

These coordinates did not work: 55.732375266666665,12.5801958
<Response [504]>

Coordinates: 55.732375266666665,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.732375266666665,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.732375266666665,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.732375266666665,12.5833506 is to low and in the dangerzone of flooding

These coordinates did not work: 55.732375266666665,12.5838764
<Response [504]>

Coordinates: 55.732375266666665,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7325808,12.577041
<Response [504]>

These coordinates did not work: 55.7325808,12.5801958
<Response [504]>

These coordinates did not work: 55.7325808,12.5807216
<Response [504]>

These coordinates did not work: 55.7325808,12.581247399999999
<Response [504]>

Coordinates: 55.7325808,12.581773199999999 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7325808,12.582298999999999
<Response [504]>

Coordinates: 55.7325808,12.5828248 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7325808,12.5833506
<Response [504]>

These coordinates did not work: 55.7325808,12.5838764
<Response [504]>

These coordinates did not work: 55.7325808,12.5844022
<Response [504]>

These coordinates did not work: 55.73278633333334,12.577041
<Response [504]>

These coordinates did not work: 55.73278633333334,12.57967
<Response [504]>

These coordinates did not work: 55.73278633333334,12.5801958
<Response [504]>

These coordinates did not work: 55.73278633333334,12.5807216
<Response [504]>

These coordinates did not work: 55.73278633333334,12.581773199999999
<Response [504]>

Coordinates: 55.73278633333334,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.73278633333334,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.73278633333334,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.73278633333334,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.73278633333334,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.732991866666666,12.577041
<Response [504]>

These coordinates did not work: 55.732991866666666,12.581773199999999
<Response [504]>

These coordinates did not work: 55.732991866666666,12.582298999999999
<Response [504]>

Coordinates: 55.732991866666666,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.732991866666666,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.732991866666666,12.5838764 is to low and in the dangerzone of flooding

These coordinates did not work: 55.732991866666666,12.5844022
<Response [504]>

These coordinates did not work: 55.7331974,12.57967
<Response [504]>

These coordinates did not work: 55.7331974,12.5801958
<Response [504]>

These coordinates did not work: 55.7331974,12.581773199999999
<Response [504]>

Coordinates: 55.7331974,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.7331974,12.5828248 is to low and in the dangerzone of flooding

Coordinates: 55.7331974,12.5833506 is to low and in the dangerzone of flooding

These coordinates did not work: 55.7331974,12.5838764
<Response [504]>

These coordinates did not work: 55.7331974,12.5844022
<Response [504]>

These coordinates did not work: 55.73340293333334,12.5775668
<Response [504]>

These coordinates did not work: 55.73340293333334,12.57967
<Response [504]>

These coordinates did not work: 55.73340293333334,12.5801958
<Response [504]>

Coordinates: 55.73340293333334,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.73340293333334,12.582298999999999 is to low and in the dangerzone of flooding

These coordinates did not work: 55.73340293333334,12.5828248
<Response [504]>

Coordinates: 55.73340293333334,12.5833506 is to low and in the dangerzone of flooding

Coordinates: 55.73340293333334,12.5838764 is to low and in the dangerzone of flooding

Coordinates: 55.73340293333334,12.5844022 is to low and in the dangerzone of flooding

These coordinates did not work: 55.73360846666667,12.577041
<Response [502]>

These coordinates did not work: 55.73360846666667,12.5780926
<Response [504]>

These coordinates did not work: 55.73360846666667,12.5786184
<Response [502]>

These coordinates did not work: 55.73360846666667,12.57967
<Response [504]>

These coordinates did not work: 55.73360846666667,12.5807216
<Response [504]>

These coordinates did not work: 55.73360846666667,12.581247399999999
<Response [504]>

Coordinates: 55.73360846666667,12.581773199999999 is to low and in the dangerzone of flooding

Coordinates: 55.73360846666667,12.582298999999999 is to low and in the dangerzone of flooding

Coordinates: 55.73360846666667,12.5828248 is to low and in the dangerzone of flooding

These coordinates did not work: 55.73360846666667,12.5833506
<Response [504]>

These coordinates did not work: 55.73360846666667,12.5838764
<Response [504]>

These coordinates did not work: 55.73360846666667,12.5844022
<Response [504]>

These coordinates did not work: 55.73074470222222,12.5775668
<Response [504]>

These coordinates did not work: 55.73078580888889,12.5791442
<Response [504]>

These coordinates did not work: 55.73096393777778,12.5780926
<Response [504]>

These coordinates did not work: 55.731059853333335,12.581773199999999
<Response [504]>

These coordinates did not work: 55.73114206666666,12.577041
<Response [504]>

These coordinates did not work: 55.731183173333335,12.5786184
<Response [502]>

These coordinates did not work: 55.731320195555554,12.5838764
<Response [504]>

These coordinates did not work: 55.7313476,12.577041
<Response [504]>

These coordinates did not work: 55.73136130222222,12.5775668
<Response [504]>

These coordinates did not work: 55.731388706666664,12.5786184
<Response [504]>

These coordinates did not work: 55.73147092,12.581773199999999
<Response [504]>

These coordinates did not work: 55.73148462222222,12.582298999999999
<Response [504]>

These coordinates did not work: 55.731498324444445,12.5828248
<Response [504]>

These coordinates did not work: 55.731512026666664,12.5833506
<Response [504]>

These coordinates did not work: 55.73158053777778,12.5780926
<Response [504]>

These coordinates did not work: 55.731635346666664,12.5801958
<Response [504]>

These coordinates did not work: 55.73164904888889,12.5807216
<Response [504]>

These coordinates did not work: 55.731676453333336,12.581773199999999
<Response [504]>

These coordinates did not work: 55.73171756,12.5833506
<Response [502]>

These coordinates did not work: 55.73173126222222,12.5838764
<Response [504]>

These coordinates did not work: 55.731758666666664,12.577041
<Response [504]>

These coordinates did not work: 55.73177236888889,12.5775668
<Response [504]>

These coordinates did not work: 55.731799773333336,12.5786184
<Response [504]>

These coordinates did not work: 55.731813475555555,12.5791442
<Response [504]>

These coordinates did not work: 55.731881986666664,12.581773199999999
<Response [504]>

These coordinates did not work: 55.73190939111111,12.5828248
<Response [504]>

These coordinates did not work: 55.731923093333336,12.5833506
<Response [504]>

These coordinates did not work: 55.7319642,12.577041
<Response [504]>

These coordinates did not work: 55.731991604444445,12.5780926
<Response [504]>

These coordinates did not work: 55.732005306666665,12.5786184
<Response [504]>

These coordinates did not work: 55.732060115555555,12.5807216
<Response [504]>

These coordinates did not work: 55.732114924444446,12.5828248
<Response [504]>

These coordinates did not work: 55.732251946666665,12.5801958
<Response [504]>

These coordinates did not work: 55.73240267111111,12.5780926
<Response [504]>

These coordinates did not work: 55.732416373333336,12.5786184
<Response [504]>

These coordinates did not work: 55.73269041777778,12.581247399999999
<Response [504]>

These coordinates did not work: 55.73290965333334,12.581773199999999
<Response [504]>

These coordinates did not work: 55.73293705777778,12.5828248
<Response [504]>

These coordinates did not work: 55.73295076,12.5833506
<Response [504]>

These coordinates did not work: 55.73296446222222,12.5838764
<Response [504]>

These coordinates did not work: 55.73306037777778,12.57967
<Response [504]>

These coordinates did not work: 55.73315629333334,12.5833506
<Response [504]>

These coordinates did not work: 55.733169995555556,12.5838764
<Response [504]>

These coordinates did not work: 55.73321110222222,12.5775668
<Response [504]>

These coordinates did not work: 55.73351255111111,12.581247399999999
<Response [504]>

These coordinates did not work: 55.73358106222222,12.5838764
<Response [504]>

These coordinates did not work: 55.73363587111111,12.5780926
<Response [504]>

These coordinates did not work: 55.73364957333334,12.5786184
<Response [504]>

These coordinates did not work: 55.73370438222222,12.5807216
<Response [504]>